<div style="direction:rtl;line-height:300%;">
	<font face="XB Zar" size=5>
		<div align=center>
			<font face="IranNastaliq" size=30>
				<p></p>
				<p></p>
به نام خدا
				<p></p>
			</font>
			<font color=#FF7500>
دانشگاه صنعتی شریف - دانشکده مهندسی کامپیوتر
            </font>
			<p></p>
			<font color=blue>
هوش مصنوعی
            </font>
			<br />
			<br />
پاییز ۱۳۹۹
		</div>
		<div align=center>
		    <font color=#FF7500 size=6>
			    <br />
تمرین ششم، بخش دوم
                سوال عملی
            	<br/>
			</font>
        </div>
	</font>
    <hr/>
</div>

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=3>
در این تمرین با پیمودن گام‌های مشخص شده یک شبکه عصبی دولایه با معماری ساده پیاده‌سازی می‌کنید. در ابتدا با اجرا کردن قطعه کد تنظیمات اولیه انجام می‌شوند. در هنگام انجام تمرین و مراجعه به فایل کد در آن قسمت‌های TODO گذاشته شده که شما باید آن قسمت‌ها را پر کنید و توضیحاتی علاوه بر توضیحات این فایل jupyter نیز در آنجا داده شده.<br>
        این تمرین برگرفته از تمرین‌های درس cs231n می‌باشد.
	</font>
</div>

In [ ]:
# A bit of setup

import numpy as np
import matplotlib.pyplot as plt

from neural_net import TwoLayerNet

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=3>
        کلاس 
   TwoLayerNet در فایل neural_net.py متعلق به مدلی که قصد طراحی آن را دارید می‌باشد که کد‌های خود را در این فایل پیاده سازی می‌کنید.
    پارامتر‌های شبکه در self.params ذخیره می‌شوند. کد زیر به تست کردن کد شما در حین پیاده‌سازی کمک می‌کند بنابراین بدون ایجاد تغییر در بلوک زیر آن را اجرا کنید.
        <br>
        معماری این شبکه به صورت زیر می‌باشد:<br>
        <ul>
            <li>ورودی دارای D ویژگی می‌باشد سپس لایه‌ی مخفی دارای H عدد پرسپترون می‌باشد. این شبکه در نهایت مقدار c مقدار خروجی می‌دهد که تعداد کلاس‌هایی که می‌خواهیم شناسایی کنیم می‌باشد. دقت کنید که این سه مقدار به شبکه به عنوان ورودی داده می‌شود و بر اساس آن شبکه ساخته می‌شود.</li>
            <li>
                در این شبکه بعد از لایه‌ی اول از تابع فعال‌سازی relu استفاده می‌کنیم و در نهایت برای محاسبه کردن هزینه‌ی شبکه عصبی با توجه به اینکه c مقدار خروجی داریم از تابع هزینه‌ی softmax استفاده می‌کنیم که در ادامه با آن آشنا خواهید شد.
                $$input 🡒 hidden\, layer 🡒 relu 🡒 output\, layer 🡒 softmax$$
            </li>
        </ul>
	</font>
</div>

In [ ]:
# Create a small net and some toy data to check your implementations.
# Note that we set the random seed for repeatable experiments.

input_size = 4
hidden_size = 10
num_classes = 3
num_inputs = 5

def init_toy_model():
    np.random.seed(0)
    return TwoLayerNet(input_size, hidden_size, num_classes, std=1e-1)

def init_toy_data():
    np.random.seed(1)
    X = 10 * np.random.randn(num_inputs, input_size)
    y = np.array([0, 1, 2, 2, 1])
    return X, y

net = init_toy_model()
X, y = init_toy_data()

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=3>
    <h3>محاسبه‌ی امتیازات</h3>
        در این مرحله باید قسمتی از تابع loss در کلاس TwoLayerNet را پیاده کنید(تا جایی که امتیازهای مربوط به هر کلاس بدست می‌آیند) در مرحله‌ی بعد امتیازات محاسبه شده توسط شما به تابع softmax loss داده می‌شود تا هزینه‌ی این شبکه بدست آید.
        <br>
        پس از اجرای بلوک زیر مقدار بدست آمده توسط کد شما و مقدار اصلی مقایسه می‌شوند و تفاوت آن‌ها نمایش داده می‌شود. این مقدار باید بسیار کوچک باشد (در حد $ 10^{-8}$)
    <br> به نکات کلی زیر نیز توجه کنید:
        <ul>
            <li>
                w1 وزن‌های مربوط به لایه‌ی اول می‌باشد که ماتریس از با شکل (D,H) می‌باشد که D تعداد ویژگی‌های ورودی‌ها و H تعداد پرسپترون‌های لایه‌ی مخفی می‌باشد و مقدار آن در self.params["w1"] ذخیره شده است.
            </li>
            <li>
                b1 برای بایاس‌های مربوط به لایه‌ی اول می‌باشد که دارای شکل (,H) می‌باشد که مقدار آن در self.params["b1"] دخیره شده است.
            </li>
            <li>
                w2 ماتریس وزن‌ها برای لایه‌ی خروجی می‌باشد که دارای شکل (H,C) می‌باشد که مقدار آن در self.params["w2"] ذخیره شده است.
            </li>
            <li>
                b2 ماتریس مربوط به بایاس‌های لایه‌ی خروجی می‌باشد که دارای شکل (,C) می‌باشد که مقدار آن در self.params["b2"] ذخیره شده است.
            </li>
            <li>
                H,C,D ورودی‌های مدل می‌باشند.
            </li>
        </ul>
        نکات زیر نیز درباره ورودی و خروجی تابع مهم می‌باشند:
        <ul>
            <li>
                ورودی x داده‌های ورودی به شبکه می‌باشند که یک ماتریس با ابعاد (N,D) می‌باشند.
            </li>
            <li>
                ورودی y که یک ماتریس با ابعاد (,N) می‌باشد نشان دهنده‌ی کلاس درست مربوط به ورودی‌های x می‌باشد. این ورودی می‌تواند به تابع داده شود یا نشود در صورتی که داده نشود این تابع فقط امتیازات را محاسبه می‌کند و خروجی می‌دهد و در صورتی که داده شود با استفاده از آن‌ها و امتیازات بدست آمده مقادیر گرادیان و هزینه را نیز محاسبه می‌کند و آن‌ها را خروجی می‌دهد.
            </li>
        </ul>
    </font>
    
</div>

In [ ]:
scores = net.loss(X)
print('Your scores:')
print(scores)
print()
print('correct scores:')
correct_scores = np.asarray([
  [-0.81233741, -1.27654624, -0.70335995],
  [-0.17129677, -1.18803311, -0.47310444],
  [-0.51590475, -1.01354314, -0.8504215 ],
  [-0.15419291, -0.48629638, -0.52901952],
  [-0.00618733, -0.12435261, -0.15226949]])
print(correct_scores)
print()

# The difference should be very small. We get < 1e-7
print('Difference between your scores and correct scores:')
print(np.sum(np.abs(scores - correct_scores)))

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=3>
    <h3>محاسبه‌ی هزینه</h3>
    در این مرحله هزینه شبکه عصبی با استفاده از امتیازاتی که در مرحله‌ی قبل بدست آوردید توسط تابع هزینه‌ی softmax محاسبه می‌شود (همچنان در همان تابع loss که در مرحله قبل در حال پر کردن قسمتی از آن بودید این کار را انجام می‌دهید). در این مرحله شما صرفا باید به محاسبات انجام شده مقدار مربوط به regularization را اضافه کنید.(مقدار loss که محاسبه شده است را باید آپدیت کنید) تابع softmax به صورت زیر است:(مقدار تفاوت باید حدودا $10^{-13}$ باشد) 
    </font>
</div>


$$L_i=-\log{(\frac{e^{f_{y_i}}}{\sum_{k}{e^{f_{yk}}}})}$$


In [ ]:
loss, _ = net.loss(X, y, reg=0.05)
correct_loss = 1.30378789133

# should be very small, we get < 1e-12
print('Difference between your loss and correct loss:')
print(np.sum(np.abs(loss - correct_loss)))

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=3>
    <h3>محاسبه‌ی گرادیان</h3>
    در این بخش به محاسبه‌ی گرادیان ها می‌پردازیم. این گرادیان‌ها را برای بروزرسانی کردن w1, w2, b1, b2 می‌باشد تا هزینه‌ی شبکه‌ی عصبی کاهش یابد و خطای آن کمتر شود. برای انجام محاسبات دقت کنید که گرادیان قسمت softmax محاسبه شده و در dscores قرار گرفته و شما باید با استفاده از آن و با کمک back-propagation گرادیان‌های W1,W2,b1,b2 را محاسبه کنید.<br>
        برای پیاده‌سازی این قسمت در ادامه‌ی همان تابع loss این کار را انجام دهید و در قسمت مربوطه پیاده‌سازی کنید. دقت شود که مقادیر بدست آمده را باید در یک dictionary برگردانید که key همان مقدار می‌باشد و value باید ماتریس گرادیان را قرار دهید دقت شود که ابعاد ماتریس گرادیان باید مانند خود ماتریس وزن یا بایاس‌ها باشد. همچنین در محاسبات دخالت دادن L2 Regularization را فراموش نکنید.(خطای محاسبات باید حدودا برابر با $10^{-9}$ باشد) 
</font>
</div>

In [ ]:
from gradient_check import eval_numerical_gradient

# Use numeric gradient checking to check your implementation of the backward pass.
# If your implementation is correct, the difference between the numeric and
# analytic gradients should be less than 1e-8 for each of W1, W2, b1, and b2.

loss, grads = net.loss(X, y, reg=0.05)

# these should all be less than 1e-8 or so
for param_name in grads:
    f = lambda W: net.loss(X, y, reg=0.05)[0]
    param_grad_num = eval_numerical_gradient(f, net.params[param_name], verbose=False)
    print('%s max relative error: %e' % (param_name, rel_error(param_grad_num, grads[param_name])))

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=3>
    <h3>آموزش شبکه عصبی</h3>
        در این قسمت برای آموزش شبکه عصبی از stochastic gradient descent استفاده می‌کنیم. برای اینکار باید تابع train در مدل را پیاده‌سازی کنید. علاوه بر تابع train باید تابع predict را نیز پیاده کنید زیرا در حین آموزش برای محاسبه‌ی دقت از این تابع استفاده می‌شود. پس از پیاده‌سازی بلوک زیر را اجرا کنید مقدار هزینه باید کمتر از 0.02 شود.
</font>
</div>

In [ ]:
net = init_toy_model()
stats = net.train(X, y, X, y,
            learning_rate=1e-1, reg=5e-6,
            num_iters=100, verbose=False)

print('Final training loss: ', stats['loss_history'][-1])

# plot the loss history
plt.plot(stats['loss_history'])
plt.xlabel('iteration')
plt.ylabel('training loss')
plt.title('Training Loss history')
plt.show()

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=3>
    <h3>خواندن داده‌ها</h3>
    حال که پیاده‌سازی قسمت‌های بالا تمام شد با اجرای بلوک زیر داده‌ها که در پوشه datasets قرار داده شده خوانده می‌شوند و در مراحل بعد بر روی آن‌ها شبکه را آموزش می‌دهیم.
</font>
</div>


In [ ]:
from data_utils import load_CIFAR10

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'datasets/cifar-10-batches-py'
    
    # Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
    try:
        del X_train, y_train
        del X_test, y_test
        print('Clear previously loaded data.')
    except:
        pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
        
    # Subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    # Reshape data to rows
    X_train = X_train.reshape(num_training, -1)
    X_val = X_val.reshape(num_validation, -1)
    X_test = X_test.reshape(num_test, -1)

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

<div dir = "rtl" style="direction:rtl;line-height:200%;">
	<font face="XB Zar" size=3>
    <h3>یادگیری</h3>
     با اجرای بلوک زیر یک شبکه با پارامتر‌های مشخص شده آموزش داده می‌شود. دقت این شبکه باید نسبتا پایین باشد.(حدود ۴۸ درصد) دلیل این می‌باشد که این شبکه برای این مجموعه داده آسان می‌باشد (دقت این شبکه به نسبت شبکه‌های آسان‌تر مانند knn یا شبکه‌ی یک لایه‌ی عادی بالاتر می‌باشد. برای اینکه این دقت افزایش باید از شبکه‌های پیچیده‌تر که از سطح این درس بالاتر می‌باشند استفاده کرد (مانند شبکه‌های چند لایه و یا CNNها) که پیشنهاد می‌شود برای مطالعه بیشتر به آن‌ها مراجعه کنید.<br>
        به نکات زیر توجه درباره‌ی این تابع train توجه کنید:
        <ul>
            <li>X: یک آرایه‌ی numpy با ابعاد (N,D) که داده‌های قسمت training را فراهم می‌کند.</li>
            <li>Y: یک آرایه‌ی numpy با ابعاد (,N) که جواب‌های مربوط به قسمت training را فراهم می‌کند که سطر iام آن به این معناست که کلاس سطر iام X چه می‌باشد.</li>
            <li>X_val: مانند X فقط برای قسمت validation استفاده می‌شود.</li>
            <li>Y_val: مانند Y فقط باری قسمت validation استفاده می‌شود.</li>
            <li>learning_rate: پارامتر مربوط سرعت آموزش می‌باشد.</li>
            <li>learning_rate_decay: بعد از هر epoch پارامتر مربوط به سرعت یادگیری در این مقدار ضرب می‌شود و کوچکتر می‌شود.</li>
            <li>reg: پارامتر مربوط به regularization می‌باشد.</li>
            <li>num_iters: تعداد iteration هایی که برای عمل بهینه‌سازی مورد استفاده قرار می‌گیرد.</li>
            <li>verbos: مقدار بولین که تصمیم می‌گیرد در حین اجرای این تابع اطلاعت چاپ شوند یا نه.</li>
        </ul>
        همانطور که گفته شد در این قسمت باید تابع predict را نیز پیاده‌سازی کنید به نکات زیر در مورد آن دقت کنید:
        <ul>
            <li> در این تابع باید با استفاده از وزن‌های یادگرفته شده پیشبینی‌های خود را درباره‌ی ورودی‌ها برگردانید.</li>
            <li>X: ورودی با ابعاد (N,D) می‌باشد.</li>
            <li>y_pred: خروجی تابع می‌باشد که پیشبینی‌های مربوط به ورودی X می‌باشد و سایز آن برابر با (,N) می‌باشد.</li>
        </ul>
</font>
</div>

In [ ]:
input_size = 32 * 32 * 3
hidden_size = 200
num_classes = 10
net = TwoLayerNet(input_size, hidden_size, num_classes)

# Train the network
stats = net.train(X_train, y_train, X_val, y_val,
            num_iters=1000, batch_size=200,
            learning_rate=1.0e-03, learning_rate_decay=0.95,
            reg=0.15, verbose=True)

# Predict on the validation set
val_acc = (net.predict(X_val) == y_val).mean()
print('Validation accuracy: ', val_acc)
